In [34]:
def read_output_file(file_path):
    paths = []  # 用于存储所有路径数据
    missing_net_ids = []  # 用于存储没有路径的 net_id
    with open(file_path, 'r') as file:
        net_number = int(file.readline().strip())
        for _ in range(net_number):
            net_id = int(file.readline().strip())
            path = []
            line = file.readline().strip()
            if line == '0':
                missing_net_ids.append(net_id)
            else:
                while line != '0':
                    layer_info, x_coord, y_coord = map(int, line.split())
                    path.append({'layer': layer_info, 'x': x_coord, 'y': y_coord})
                    line = file.readline().strip()
                paths.append({'net_id': net_id, 'path': path})  # 将路径数据添加到 paths 列表中
    if missing_net_ids:
        for net_id in missing_net_ids:
            print(f"没有布线的节点: {net_id}")
    else:
        print("所有的节点均已布线.")
    return paths, missing_net_ids

# Usage example
file_path = '../output/test.router'
paths,missing_net_ids = read_output_file(file_path)

没有布线的节点: 2


In [15]:
paths[1]

[2]

## 检查重复点

In [18]:
def check_duplicate_points(output_data):
    visited_points = set()
    duplicate_points = {}
    for net in output_data:
        net_id = net['net_id']
        for point in net['path']:
            x = point['x']
            y = point['y']
            layer = point['layer']
            point_tuple = (x, y, layer)
            if point_tuple in visited_points:
                if point_tuple in duplicate_points:
                    duplicate_points[point_tuple].append(net_id)
                else:
                    duplicate_points[point_tuple] = [net_id]
            else:
                visited_points.add(point_tuple)
    
    if len(duplicate_points) > 0:
        print("存在重复的点:")
        for point, net_ids in duplicate_points.items():
            print(f"X: {point[0]}, Y: {point[1]}, Layer: {point[2]}")
            print("重复的Net ID:", net_ids)
    else:
        print("未找到重复的点。")
    
    return duplicate_points

# 使用示例
duplicate_points = check_duplicate_points(paths)



存在重复的点:
X: 15, Y: 25, Layer: 2
重复的Net ID: [4]


## 检查每条线是否连续

In [21]:
def check_adjacent_points(path):
    for i in range(len(path) - 1):
        current_point = path[i]
        next_point = path[i + 1]
        diff_count = 0
        if current_point['x'] != next_point['x']:
            diff_count += 1
        if current_point['y'] != next_point['y']:
            diff_count += 1
        if current_point['layer'] != next_point['layer']:
            diff_count += 1
        if diff_count != 1:
            return False
    return True

def check_adjacent_paths(paths):
    incorrect_paths = []  # 存储不正确的路径的 ID
    for data in paths:
        path = data['path']
        is_adjacent = check_adjacent_points(path)
        if not is_adjacent:
            incorrect_paths.append(data['net_id'])
    if len(incorrect_paths) > 0:
        print("不连续布线的Net ID:",incorrect_paths)
    else:
        print("所有布线都连续")
    return

In [22]:
check_adjacent_paths(paths)

不连续布线的Net ID: [4]


## 检查每条布线的起点和终点是否正确

In [38]:
def check_path_coordinates(paths, nets):
    matching_paths = []  # 用于存储匹配的路径数据
    for path_data in paths:
        net_id = path_data['net_id']
        path = path_data['path']
        net = next((net for net in nets if net['net_id'] == net_id), None)
        if net is None:
            print(f"无法找到匹配的网络列表数据,Net ID: {net_id}")
        else:
            pin1 = net['pin1']
            pin2 = net['pin2']
            path_start = path[0]
            path_end = path[-1]
            if (path_start['layer'], path_start['x'], path_start['y']) != (pin1['layer'], pin1['x'], pin1['y']):
                print(f"路径起点与网络列表不匹配，Net ID: {net_id}")
            #else:
            #    print(f"路径起点与网络列表匹配，Net ID: {net_id}")
            if (path_end['layer'], path_end['x'], path_end['y']) != (pin2['layer'], pin2['x'], pin2['y']):
                print(f"路径终点与网络列表不匹配，Net ID: {net_id}")
            else:
                matching_paths.append(path_data)  # 添加匹配的路径数据到列表中

    if len(matching_paths) == len(paths):
        print("布线结果的起点和终点与Nets全部匹配")
    else:
        print("布线结果的起点和终点与Nets发现不匹配")
    
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
netlist_file_path = '../benchmark/bench4.nl'
nets,net_num = parse_netlist(netlist_file_path)
check_path_coordinates(paths, nets)

路径起点与网络列表不匹配，Net ID: 1
路径终点与网络列表不匹配，Net ID: 15
布线结果的起点和终点与Net发现不匹配


# 计算cost

In [48]:
def calculate_path_cost(paths, bend_penalty, via_penalty, layer1_grid, layer2_grid):
    total_cost = 0
    cost_path = {}
    for path in paths:
        net_id = path['net_id']
        path_data = path['path']
        cost_path[net_id] = 0
        cost = 0
        # 计算每个点的成本
        for i, point in enumerate(path_data):
            
            x = point['x']
            y = point['y']
            layer = point['layer']

            if layer == 1 or layer == 2:
                # 加上 layer1/2_grid 上对应 (x, y) 的值
                cost += layer1_grid[x][y] if layer == 1 else layer2_grid[x][y]
            elif layer == 3:
                cost += via_penalty
            if i > 1:
                # 计算路径弯曲的成本
                prev_prev_layer = path_data[i - 2]['layer']
                prev_prev_x = path_data[i - 2]['x']
                prev_prev_y = path_data[i - 2]['y']
                prev_layer = path_data[i - 1]['layer']
                if prev_prev_layer == layer and prev_layer == layer and (prev_prev_x != x and y != prev_prev_y):
                    cost += bend_penalty
        cost_path[net_id]=cost
        total_cost += cost
                        
    return cost_path,total_cost

In [49]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
filepath_out='../output/bench4.router'
netlist_file_path = '../benchmark/bench4.nl'
gridfile_path  = '../benchmark/bench4.grid'
nets,net_num = parse_netlist(netlist_file_path)
rows,columns,bend_penalty,via_penalty,layer1_grid,layer2_grid= parse_gridfile(gridfile_path)
cost_path,total_cost=calculate_path_cost(paths, bend_penalty, via_penalty, layer1_grid.T, layer2_grid.T)

bend_penalty: 50 
 via_penalty: 50
size of layer1: (50, 50)
size of layer2: (50, 50)


In [51]:
total_cost

1724.0

In [50]:
cost_path


{1: 64.0,
 3: 69.0,
 4: 122.0,
 5: 124.0,
 6: 126.0,
 7: 193.0,
 8: 193.0,
 9: 63.0,
 10: 65.0,
 11: 118.0,
 12: 120.0,
 13: 122.0,
 14: 173.0,
 15: 172.0}

# 集成

In [69]:
def read_output_file(file_path):
    paths = []  # 用于存储所有路径数据
    missing_net_ids = []  # 用于存储没有路径的 net_id
    with open(file_path, 'r') as file:
        net_number = int(file.readline().strip())
        for _ in range(net_number):
            net_id = int(file.readline().strip())
            path = []
            line = file.readline().strip()
            if line == '0':
                missing_net_ids.append(net_id)
            else:
                while line != '0':
                    layer_info, x_coord, y_coord = map(int, line.split())
                    path.append({'layer': layer_info, 'x': x_coord, 'y': y_coord})
                    line = file.readline().strip()
                paths.append({'net_id': net_id, 'path': path})  # 将路径数据添加到 paths 列表中
    if missing_net_ids:
        for net_id in missing_net_ids:
            print(f"没有布线的节点: {net_id}")
    else:
        print("所有的节点均已布线.")
    return paths, missing_net_ids

def check_duplicate_points(output_data):
    visited_points = set()
    duplicate_points = {}
    for net in output_data:
        net_id = net['net_id']
        for point in net['path']:
            x = point['x']
            y = point['y']
            layer = point['layer']
            point_tuple = (x, y, layer)
            if point_tuple in visited_points:
                if point_tuple in duplicate_points:
                    duplicate_points[point_tuple].append(net_id)
                else:
                    duplicate_points[point_tuple] = [net_id]
            else:
                visited_points.add(point_tuple)
    
    if len(duplicate_points) > 0:
        print("存在重复的点:")
        for point, net_ids in duplicate_points.items():
            print(f"X: {point[0]}, Y: {point[1]}, Layer: {point[2]}")
            print("重复的Net ID:", net_ids)
    else:
        print("未找到重复的点。")
    
    return duplicate_points


def check_adjacent_points(path):
    for i in range(len(path) - 1):
        current_point = path[i]
        next_point = path[i + 1]
        diff_count = 0
        if current_point['x'] != next_point['x']:
            diff_count += 1
        if current_point['y'] != next_point['y']:
            diff_count += 1
        if current_point['layer'] != next_point['layer']:
            diff_count += 1
        if diff_count != 1:
            return False
    return True

def check_adjacent_paths(paths):
    incorrect_paths = []  # 存储不正确的路径的 ID
    for data in paths:
        path = data['path']
        is_adjacent = check_adjacent_points(path)
        if not is_adjacent:
            incorrect_paths.append(data['net_id'])
    if len(incorrect_paths) > 0:
        print("不连续布线的Net ID:",incorrect_paths)
    else:
        print("所有布线都连续")
    return


def check_path_coordinates(paths, nets):
    matching_paths = []  # 用于存储匹配的路径数据
    for path_data in paths:
        net_id = path_data['net_id']
        path = path_data['path']
        net = next((net for net in nets if net['net_id'] == net_id), None)
        if net is None:
            print(f"无法找到匹配的网络列表数据,Net ID: {net_id}")
        else:
            pin1 = net['pin1']
            pin2 = net['pin2']
            path_start = path[0]
            path_end = path[-1]
            if (path_start['layer'], path_start['x'], path_start['y']) != (pin1['layer'], pin1['x'], pin1['y']):
                print(f"路径起点与网络列表不匹配，Net ID: {net_id}")
            #else:
            #    print(f"路径起点与网络列表匹配，Net ID: {net_id}")
            if (path_end['layer'], path_end['x'], path_end['y']) != (pin2['layer'], pin2['x'], pin2['y']):
                print(f"路径终点与网络列表不匹配，Net ID: {net_id}")
            else:
                matching_paths.append(path_data)  # 添加匹配的路径数据到列表中

    if len(matching_paths) == len(paths):
        print("布线结果的起点和终点与Nets全部匹配")
    else:
        print("布线结果的起点和终点与Nets发现不匹配")
        
def calculate_path_cost(paths, bend_penalty, via_penalty, layer1_grid, layer2_grid):
    total_cost = 0
    cost_path = {}
    for path in paths:
        net_id = path['net_id']
        path_data = path['path']
        cost_path[net_id] = 0
        cost = 0
        # 计算每个点的成本
        for i, point in enumerate(path_data):
            
            x = point['x']
            y = point['y']
            layer = point['layer']

            if layer == 1 or layer == 2:
                # 加上 layer1/2_grid 上对应 (x, y) 的值
                cost += layer1_grid[x][y] if layer == 1 else layer2_grid[x][y]
            elif layer == 3:
                cost += via_penalty
            if i > 1:
                # 计算路径弯曲的成本
                prev_prev_layer = path_data[i - 2]['layer']
                prev_prev_x = path_data[i - 2]['x']
                prev_prev_y = path_data[i - 2]['y']
                prev_layer = path_data[i - 1]['layer']
                if prev_prev_layer == layer and prev_layer == layer and (prev_prev_x != x and y != prev_prev_y):
                    cost += bend_penalty
        cost_path[net_id]=cost
        total_cost += cost
    print("total_cost:",total_cost)               
    return cost_path,total_cost

In [71]:
from parase_input_package.generate_output import *
from parase_input_package.parase_input import *
from parase_input_package.plot_output import *
router_path = '../output/test.router'
bench_name ="bench4"
def evaluate_route(router_path,bench_name):
    paths,_ = read_output_file(router_path)
    duplicate_points = check_duplicate_points(paths)
    check_adjacent_paths(paths)
    netlist_file_path = '../benchmark/'+bench_name+'.nl'
    gridfile_path  = '../benchmark/'+bench_name+'.grid'
    nets,_ = parse_netlist(netlist_file_path)
    _,_,bend_penalty,via_penalty,layer1_grid,layer2_grid = parse_gridfile(gridfile_path)
    check_path_coordinates(paths, nets)
    cost_path,total_cost=calculate_path_cost(paths, bend_penalty, via_penalty, layer1_grid.T, layer2_grid.T)
    return duplicate_points ,cost_path,total_cost
  
duplicate_points ,cost_path,total_cost=evaluate_route(router_path,bench_name)  

没有布线的节点: 2
存在重复的点:
X: 15, Y: 25, Layer: 2
重复的Net ID: [4]
不连续布线的Net ID: [4]
bend_penalty: 50 
 via_penalty: 50
size of layer1: (50, 50)
size of layer2: (50, 50)
路径起点与网络列表不匹配，Net ID: 1
路径终点与网络列表不匹配，Net ID: 15
布线结果的起点和终点与Nets发现不匹配
total_cost: 1724.0
